In [ ]:
!pip install recbole
!pip install ray

In [15]:
import pandas as pd
import numpy as np
import torch

import sys
import logging
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.model.sequential_recommender import SASRec, BERT4Rec
from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger
from recbole.quick_start import load_data_and_model
from recbole.data import Interaction

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Трансформирование датасета KION в формат Recbole

In [5]:
inter_df = pd.read_csv('interactions_processed.csv')

In [6]:
!mkdir "recbole_data"

In [7]:
filename = f"recbole_data/recbole_data.inter"

In [8]:
inter_df['last_watch_dt'] = pd.to_datetime(inter_df['last_watch_dt']).astype('int64') // 10**9
inter_df.drop("total_dur", axis=1, inplace=True)

inter_df.rename(columns={
    "user_id": "user_id:token", 
    "item_id": "item_id:token", 
    "last_watch_dt": "timestamp:float", 
    "watched_pct": "watched_pct:float",
}, inplace=True)

inter_df.head(3)

,user_id:token,item_id:token,timestamp:float,watched_pct:float
0,176549,9506,1620691200,72
1,699317,1659,1622246400,100
2,656683,7107,1620518400,0


In [9]:
inter_df.to_csv(filename, sep="\t", index=False)

## SASRec

In [ ]:
parameter_dict = {
    "worker": 2,
    "device": "GPU",
    "data_path": "",

    "USER_ID_FIELD": "user_id",
    "ITEM_ID_FIELD": "item_id",
    "TIME_FIELD": "timestamp",
    "load_col" : {"inter": ["user_id", "item_id", "timestamp", "watched_pct"]},

    "val_interval": {"watched_pct": "[10,inf)"},
    "user_inter_num_interval": "[40,inf)",
    "item_inter_num_interval": "[40,inf)",

    "epoch": 100,
    
    "eval_args": {
        "group_by": "user",
        "order": "TO",
        "split": {"RS": [8, 1, 1]},
        "mode": "full",
    },
    "metrics": ["MAP", "Recall", "Precision", "MRR", "NDCG", "ItemCoverage", "GiniIndex"],
    "valid_metric": "MRR@10",

    "hidden_size": 128,
    "n_layers": 2,
    "n_heads": 4,
    "hidden_act": "swish",
    "loss_type": "BPR",
    "train_neg_sample_args": {
        "distribution": "popularity",
        "sample_num": 1,

    },
}

In [ ]:
# conf inittialization
config = Config(model='SASRec', dataset='recbole_data', config_dict=parameter_dict)

# init random seed
init_seed(config["seed"], config["reproducibility"])

# logger init
init_logger(config)

logger = logging.getLogger()
logFormatter = logging.Formatter("%(asctime)s [%(levelname)-5.5s]  %(message)s")
# Create handlers
c_handler = logging.StreamHandler(sys.stdout)
c_handler.setFormatter(logFormatter)
c_handler.setLevel("INFO")

f_handler = logging.FileHandler("logs.txt")
f_handler.setFormatter(logFormatter)
f_handler.setLevel("INFO")

logger.addHandler(c_handler)
logger.addHandler(f_handler)

In [ ]:
dataset = create_dataset(config)
dataset

recbole_data
The number of users: 6096
Average actions of users: 60.29975389663659
The number of items: 2157
Average actions of items: 170.46706864564007
The number of inters: 367527
The sparsity of the dataset: 97.20492062101417%
Remain Fields: ['user_id', 'item_id', 'timestamp', 'watched_pct']

In [ ]:
train_data, valid_data, test_data = data_preparation(config, dataset)

In [ ]:
results = dict()

sasrec = SASRec(config, train_data.dataset).to(config["device"])
trainer = Trainer(config, sasrec)

# training
sas_best_valid_score, sas_best_valid_results = trainer.fit(
    train_data,
    valid_data,
)

# evaluation
sas_test_result = trainer.evaluate(test_data)

results["SASRec"] = sas_test_result

In [ ]:
results

{'SASRec': OrderedDict([('map@10', 0.0152),
              ('recall@10', 0.0497),
              ('precision@10', 0.005),
              ('mrr@10', 0.0152),
              ('ndcg@10', 0.0232),
              ('itemcoverage@10', 0.9838),
              ('giniindex@10', 0.7688)])}

## BERT4Rec

In [ ]:
# conf inittialization
config = Config(model='BERT4Rec', dataset='recbole_data', config_dict=parameter_dict)

# init random seed
init_seed(config["seed"], config["reproducibility"])

# logger init
init_logger(config)

In [ ]:
dataset = create_dataset(config)
train_data, valid_data, test_data = data_preparation(config, dataset)

In [ ]:
bert4rec = BERT4Rec(config, train_data.dataset).to(config["device"])
trainer = Trainer(config, bert4rec)

# training
bert_best_valid_score, bert_best_valid_results = trainer.fit(
    train_data,
    valid_data, 
    show_progress=True,
)

# evaluation
bert_test_result = trainer.evaluate(test_data)

results["BERT4Rec"] = bert_test_result

Evaluate   : 100%|███████████████████████████| 9/9 [00:03<00:00,  2.31it/s, GPU RAM: 5.23 G/14.75 G]


In [ ]:
df = pd.DataFrame(
    data=[
        list(results["SASRec"].values()),
        list(results["BERT4Rec"].values()),
        list(sas_best_valid_results.values()), 
        list(bert_best_valid_results.values()),
    ],
    columns=list(results["SASRec"].keys()),
    index=["SASRec (test)", "BERT4Rec (test)", "SASRec (best valid)", "BERT4Rec (best valid)"]
)
df

,map@10,recall@10,precision@10,mrr@10,ndcg@10,itemcoverage@10,giniindex@10
SASRec (test),0.0152,0.0497,0.0050,0.0152,0.0232,0.9838,0.7688
BERT4Rec (test),0.0161,0.0499,0.0050,0.0161,0.0239,0.6528,0.8559
SASRec (best valid),0.0167,0.0543,0.0054,0.0167,0.0254,0.9893,0.7360
BERT4Rec (best valid),0.0194,0.0595,0.0059,0.0194,0.0286,0.6750,0.8516


Имплементирую в сервис модель BERT4Rec.

## Предсказания BERT4Rec. Интеграция в сервис

In [11]:
config, bert4rec, dataset, train_data, valid_data, test_data = \
    load_data_and_model("BERT4Rec-May-16-2023_10-44-04.pth")

16 May 21:53    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = recbole_data
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 300
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'popularity', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'group_by': 'user', 'order': 'TO', 'split': {'RS': [8, 1, 1]}, 'mode': 'full'}
repeatable = True
metrics = ['MAP', 'Recall', 'Precision', 'MRR', 'NDCG', 'ItemCoverage', 'GiniIndex']
topk = [10]
valid_metric = MRR@10
valid_metric_bigger = True
eval_batch_size = 4096
metric_decimal_place = 4

Dataset Hyper Parameters:
field_

In [158]:
dataset.field2id_token

{'user_id': array(['[PAD]', '176549', '699317', ..., '163369', '753860', '942675'],
       dtype='<U7'),
 'item_id': array(['[PAD]', '9506', '1659', ..., '11864', '5897', '13624'],
       dtype='<U5')}

Оставим последние 50 просмотров пользователя и сделем предсказание - 10 наиболее вероятных айтемов.

In [159]:
max_sequence_len = 50

train_df = pd.DataFrame(data={
    "user_id": dataset["user_id"],
    "item_id": dataset["item_id"],
    "timestamp": dataset["timestamp"],
}).sort_values(["user_id", "timestamp"], ignore_index=True)

train_df["order"] = train_df.groupby("user_id")["item_id"].cumcount(ascending=False) + 1
train_df = train_df[train_df.order <= max_sequence_len]
train_df.drop("order", axis=1, inplace=True)

train_df = train_df.groupby(by="user_id", as_index=False)["item_id"].agg(list)

train_df["item_length"] = train_df.item_id.apply(
    lambda items: 
    len(items) if len(items) <= max_sequence_len else max_sequence_len
)

train_df.rename(columns={"item_id": "item_id_list"}, inplace=True)
train_df

,user_id,item_id_list,item_length
0,1,"[2064, 2042, 648, 2, 1906, 806, 936, 1239, 192...",50
1,2,"[289, 1661, 504, 828, 714, 943, 1, 809, 918, 1...",50
2,3,"[231, 1279, 111, 1417, 1915, 1701, 486, 1920, ...",50
3,4,"[443, 214, 445, 1668, 386, 1955, 240, 25, 366,...",50
4,5,"[41, 1642, 734, 393, 938, 773, 162, 538, 221, ...",50
...,...,...,...
6090,6091,"[788, 2102, 166, 1060, 350, 1258, 1481, 1848, ...",42
6091,6092,"[297, 824, 1051, 225, 677, 114, 240, 284, 16, ...",39
6092,6093,"[748, 1042, 246, 1974, 567, 735, 336, 302, 114...",40
6093,6094,"[938, 1686, 887, 516, 351, 485, 1904, 432, 131...",39


In [160]:
train_interaction = Interaction(train_df).to("cuda:0")

In [322]:
bert4rec.eval()

with torch.no_grad():
    scores = bert4rec.full_sort_predict(train_interaction)

scores = scores[:, 1:]
scores = scores.cpu().numpy()
scores.shape

(6095, 2156)

нулевой элемент во втором измерении scores[:, 0] - не является айтемом, 

так как всего уникальных айтемов = 2156

уникальных пользователей = 6095

Маппинги и чтение просмотренных айтемов:

In [310]:
import dill


user_id_to_token = dataset.field2id_token["user_id"]
user_id_to_uid = {int(user_id_to_token[uid]): uid for uid in range(1, len(user_id_to_token))}
uid_to_user_id = {v: k for k, v in user_id_to_uid.items()}

item_id_to_token = dataset.field2id_token["item_id"]
item_id_to_iid = {int(item_id_to_token[iid]): iid for iid in range(1, len(item_id_to_token))}
iid_to_item_id = {v: k for k, v in item_id_to_iid.items()}

with open("popular_in_category_model.dill", "rb") as f:
    user_id_to_watched_item_ids = dill.load(f)["user_to_watched_items_map"]
len(user_id_to_watched_item_ids)

962179

**Предсказания**:

In [367]:
user_id_to_bert4rec_recs = {}
k_recs = 10

for uid, u_scores in enumerate(scores):
    uid += 1
    user_id = uid_to_user_id[uid]
    
    watched_item_ids = user_id_to_watched_item_ids[user_id]

    n = k_recs + len(watched_item_ids)
    # iid уменьшенные на 1
    top_n_iids = np.flip(u_scores.argsort()[-n:], 0)
    
    answer = []
    for iid in top_n_iids:
        iid += 1
        if iid_to_item_id[iid] not in watched_item_ids:
            answer.append(iid_to_item_id[iid])
        if len(answer) == k_recs:
            break
    
    user_id_to_bert4rec_recs[user_id] = answer

len(user_id_to_bert4rec_recs)

6095

In [366]:
# оффлайн метод
import pickle


with open("user_id_to_bert4rec_recs.pickle", "wb") as f:
    pickle.dump(user_id_to_bert4rec_recs, f)

## Смотрим на рекомендации:

In [340]:
items_df = pd.read_csv("items_processed.csv")
interactions_df = pd.read_csv("interactions_processed.csv")

#### Пример 1.

Последние просмотренные 25 фильмов

In [356]:
test_user_id = int(np.random.choice(user_id_to_token[1:]))
test_item_ids = user_id_to_bert4rec_recs[test_user_id]

interactions_df[(interactions_df["user_id"] == test_user_id)] \
    .sort_values("last_watch_dt")[["user_id", "item_id", "last_watch_dt"]][-20:] \
    .merge(items_df[["item_id", "content_type", "title", "genres", "countries", "age_rating", "release_year_cat"]], on="item_id")

,user_id,item_id,last_watch_dt,content_type,title,genres,countries,age_rating,release_year_cat
0,818325,14177,2021-08-11,film,астерикс и тайное зелье,"мультфильм, фэнтези",франция,6.0,2010-2020
1,818325,2981,2021-08-14,film,академия вампиров,"детективы, драмы, триллеры, боевики, ужасы, фэ...",сша,12.0,2010-2020
2,818325,16270,2021-08-14,film,тайна коко,"мультфильм, фэнтези, приключения",сша,12.0,2010-2020
3,818325,14826,2021-08-14,film,толстяки,"драмы, комедии",испания,18.0,2000-2010
4,818325,13159,2021-08-14,film,рататуй,"мультфильм, приключения, драмы, фэнтези, комедии",сша,0.0,2000-2010
5,818325,11899,2021-08-15,film,вся правда о любви,"мелодрамы, комедии",великобритания,16.0,2000-2010
6,818325,11645,2021-08-16,film,и это пройдёт,"драмы, мультфильм",франция,0.0,2020_inf
7,818325,12268,2021-08-16,film,маэстро,"мультфильм, комедии",франция,0.0,2010-2020
8,818325,13988,2021-08-16,film,листок,"драмы, мультфильм",чехия,0.0,2020_inf
9,818325,9070,2021-08-16,film,рассказы,"драмы, комедии",россия,18.0,2010-2020


Будущие рекомендации

In [357]:
items_df[
    items_df.item_id.isin(test_item_ids)
][["content_type", "title", "genres", "countries", "age_rating", "release_year_cat"]]

,content_type,title,genres,countries,age_rating,release_year_cat
60,film,бетховен 2,"мелодрамы, семейное, комедии",сша,6.0,1990-2000
4617,film,джок,"приключения, мультфильм, вестерн, комедии",сша,0.0,2010-2020
7563,film,снежная королева 3. огонь и лед,"мультфильм, фэнтези, приключения, комедии",россия,6.0,2010-2020
8033,film,смурфики 2,"семейное, мультфильм, фэнтези, комедии",сша,0.0,2010-2020
8601,film,подводная эра,"мультфильм, приключения, комедии",сша,6.0,2010-2020
9698,film,зверополис,"приключения, мультфильм, детективы, комедии",сша,6.0,2010-2020
13063,film,смурфики,"мультфильм, фэнтези, приключения, комедии",сша,0.0,2010-2020
13064,film,мы – монстры,"ужасы, мультфильм, фэнтези, комедии",великобритания,6.0,2010-2020
13577,film,гадкий я 2,"мультфильм, приключения, фантастика, фэнтези, ...","сша, франция, япония",0.0,2010-2020
15886,film,гномео и джульетта,"мелодрамы, мультфильм, приключения, комедии","великобритания, канада, сша",0.0,2010-2020


#### Пример 2.

Последние просмотренные 25 фильмов

In [360]:
test_user_id = int(np.random.choice(user_id_to_token[1:]))
test_item_ids = user_id_to_bert4rec_recs[test_user_id]

interactions_df[(interactions_df["user_id"] == test_user_id)] \
    .sort_values("last_watch_dt")[["user_id", "item_id", "last_watch_dt"]][-20:] \
    .merge(items_df[["item_id", "content_type", "title", "genres", "countries", "age_rating", "release_year_cat"]], on="item_id")

,user_id,item_id,last_watch_dt,content_type,title,genres,countries,age_rating,release_year_cat
0,664739,1873,2021-08-11,film,секс трип,"фэнтези, комедии",сша,18.0,2010-2020
1,664739,15051,2021-08-11,film,порочная связь,драмы,"италия, франция",18.0,2020_inf
2,664739,6267,2021-08-12,film,смурфики 2,"семейное, мультфильм, фэнтези, комедии",сша,0.0,2010-2020
3,664739,12741,2021-08-13,series,шиммер и шайн,"мультсериалы, фэнтези, приключения","канада, сша",0.0,2010-2020
4,664739,973,2021-08-13,film,фиксики. большой секрет,"мультфильм, музыкальные, приключения, комедии",россия,6.0,2010-2020
5,664739,15078,2021-08-15,series,щенячий патруль,"мультсериалы, спорт, приключения, комедии","канада, сша",0.0,2010-2020
6,664739,12225,2021-08-15,series,фиксики,"мультсериалы, приключения, комедии",россия,0.0,2010-2020
7,664739,16244,2021-08-15,film,предложение (2009),"драмы, мелодрамы, комедии",сша,16.0,2000-2010
8,664739,11769,2021-08-16,film,мошенники,"мелодрамы, комедии","германия, сша",16.0,2000-2010
9,664739,468,2021-08-16,series,азбука безопасности на дороге,мультсериалы,украина,0.0,2000-2010


Будущие рекомендации

In [361]:
items_df[
    items_df.item_id.isin(test_item_ids)
][["content_type", "title", "genres", "countries", "age_rating", "release_year_cat"]]

,content_type,title,genres,countries,age_rating,release_year_cat
158,film,вперёд,"для детей, приключения, семейное, фэнтези, ком...",сша,6.0,2020_inf
809,film,анастасия,"мультфильм, приключения, драмы, мюзиклы, фэнтези",сша,6.0,1990-2000
1554,film,тайна коко,"мультфильм, фэнтези, приключения",сша,12.0,2010-2020
6990,film,рио 2,"мюзиклы, мультфильм, приключения, комедии",сша,0.0,2010-2020
7939,film,кролик питер,"мультфильм, фэнтези, приключения, комедии",сша,6.0,2010-2020
8698,film,алиса в зазеркалье,"семейное, фэнтези, приключения","великобритания, сша",12.0,2010-2020
9921,film,моана,"мультфильм, фэнтези, мюзиклы",сша,6.0,2010-2020
10721,film,красавица и чудовище,"мюзиклы, мультфильм, фэнтези, мелодрамы",сша,6.0,1990-2000
11254,film,суперсемейка 2,"фантастика, мультфильм, приключения",сша,6.0,2010-2020
13063,film,смурфики,"мультфильм, фэнтези, приключения, комедии",сша,0.0,2010-2020


#### Пример 3.

Последние просмотренные 25 фильмов

In [362]:
test_user_id = int(np.random.choice(user_id_to_token[1:]))
test_item_ids = user_id_to_bert4rec_recs[test_user_id]

interactions_df[(interactions_df["user_id"] == test_user_id)] \
    .sort_values("last_watch_dt")[["user_id", "item_id", "last_watch_dt"]][-20:] \
    .merge(items_df[["item_id", "content_type", "title", "genres", "countries", "age_rating", "release_year_cat"]], on="item_id")

,user_id,item_id,last_watch_dt,content_type,title,genres,countries,age_rating,release_year_cat
0,627400,3777,2021-08-17,series,драконы и всадники олуха,"мультсериалы, приключения, семейное, фэнтези, ...",сша,6.0,2010-2020
1,627400,10876,2021-08-17,film,гарфилд,"мультфильм, фэнтези, комедии",сша,0.0,2000-2010
2,627400,10755,2021-08-17,film,время монстров,"ужасы, фэнтези",сша,16.0,2010-2020
3,627400,13243,2021-08-17,film,головоломка,"фантастика, мультфильм, комедии",сша,6.0,2010-2020
4,627400,3182,2021-08-19,film,ральф против интернета,"мультфильм, приключения, фантастика, семейное,...",сша,6.0,2010-2020
5,627400,8070,2021-08-19,film,"приказ ""уничтожить""","историческое, военные",россия,12.0,2010-2020
6,627400,2025,2021-08-19,film,семейка крудс,"мультфильм, фэнтези",сша,0.0,2010-2020
7,627400,14196,2021-08-19,series,пять невест,"военные, комедии",россия,16.0,2010-2020
8,627400,5754,2021-08-19,film,доктор стрэндж,"боевики, фантастика, фэнтези, приключения",сша,16.0,2010-2020
9,627400,14317,2021-08-20,film,веном,"популярное, фантастика, триллеры, боевики, ужасы",сша,16.0,2010-2020


Будущие рекомендации

In [363]:
items_df[
    items_df.item_id.isin(test_item_ids)
][["content_type", "title", "genres", "countries", "age_rating", "release_year_cat"]]

,content_type,title,genres,countries,age_rating,release_year_cat
3491,film,человек-паук: возвращение домой,"боевики, популярное, фантастика, приключения",сша,16.0,2010-2020
4188,film,тачки 3,"мультфильм, комедии",сша,6.0,2010-2020
8754,film,ледниковый период 4: континентальный дрейф,"мультфильм, приключения, комедии",сша,0.0,2010-2020
9003,film,город героев,"боевики, фантастика, мультфильм, комедии",сша,6.0,2010-2020
9698,film,зверополис,"приключения, мультфильм, детективы, комедии",сша,6.0,2010-2020
9848,film,ральф,"мультфильм, фэнтези, комедии",сша,6.0,2010-2020
9921,film,моана,"мультфильм, фэнтези, мюзиклы",сша,6.0,2010-2020
10395,film,ледниковый период,"мультфильм, фэнтези, приключения, комедии",сша,0.0,2000-2010
10792,film,человек-паук: вдали от дома,"боевики, фантастика, приключения",сша,12.0,2010-2020
15352,film,холодное сердце ii,"фэнтези, мультфильм, музыкальные",сша,6.0,2010-2020


#### Пример 4.

Последние просмотренные 25 фильмов

In [364]:
test_user_id = int(np.random.choice(user_id_to_token[1:]))
test_item_ids = user_id_to_bert4rec_recs[test_user_id]

interactions_df[(interactions_df["user_id"] == test_user_id)] \
    .sort_values("last_watch_dt")[["user_id", "item_id", "last_watch_dt"]][-20:] \
    .merge(items_df[["item_id", "content_type", "title", "genres", "countries", "age_rating", "release_year_cat"]], on="item_id")

,user_id,item_id,last_watch_dt,content_type,title,genres,countries,age_rating,release_year_cat
0,944850,13865,2021-06-11,film,девятаев,"драмы, военные, приключения",россия,12.0,2020_inf
1,944850,11778,2021-06-13,film,простые сложности,"драмы, мелодрамы, комедии","сша, япония",16.0,2000-2010
2,944850,10152,2021-06-15,film,гамбит,"криминал, комедии",сша,12.0,2010-2020
3,944850,4930,2021-06-18,film,муж двух жён,комедии,великобритания,16.0,2010-2020
4,944850,9059,2021-06-20,film,ловец снов,ужасы,сша,18.0,2020_inf
5,944850,13223,2021-06-22,film,экстрасенс,"драмы, ужасы, триллеры",великобритания,16.0,2010-2020
6,944850,3999,2021-06-24,film,джиперс криперс,"ужасы, триллеры","германия, сша",16.0,2000-2010
7,944850,12356,2021-06-27,film,13 грехов,"ужасы, триллеры",сша,16.0,2010-2020
8,944850,16444,2021-07-02,film,закатать в асфальт,"боевики, драмы, триллеры, криминал","канада, сша",18.0,2010-2020
9,944850,14998,2021-07-04,film,невеста,"ужасы, триллеры",россия,16.0,2010-2020


Будущие рекомендации

In [365]:
items_df[
    items_df.item_id.isin(test_item_ids)
][["content_type", "title", "genres", "countries", "age_rating", "release_year_cat"]]

,content_type,title,genres,countries,age_rating,release_year_cat
2819,film,цена страсти,"драмы, триллеры, мелодрамы","германия, сша",16.0,2010-2020
3158,film,американец,"драмы, триллеры","великобритания, сша",18.0,2010-2020
3596,film,вне поля зрения,"криминал, драмы, триллеры, мелодрамы, комедии",сша,16.0,1990-2000
4344,film,прочь,"ужасы, триллеры, детективы","сша, япония",18.0,2010-2020
4537,film,кривая линия,"ужасы, триллеры",сша,18.0,2010-2020
7716,film,человек-невидимка,"фантастика, ужасы, триллеры, детективы","австралия, великобритания, канада, сша",18.0,2020_inf
8777,film,госфорд парк,"драмы, криминал, детективы, комедии","великобритания, италия, сша",16.0,2000-2010
9255,film,мёрзлая земля,"драмы, триллеры",сша,16.0,2010-2020
10172,film,наблюдатель,"драмы, триллеры, криминал, детективы",сша,18.0,2000-2010
14160,film,предместье,"детективы, комедии",сша,12.0,1980-1990
